# Umgebung vorbereiten

In [1]:
  # Repo klonen

!git clone https://github.com/sleuoth/ABDA2019.git

Cloning into 'ABDA2019'...
remote: Enumerating objects: 424, done.
remote: Total 424 (delta 0), reused 0 (delta 0), pack-reused 424
Receiving objects: 100% (424/424), 486.44 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (418/418), done.


In [2]:
# Das ausführen, wenn die Dateien aus dem Repo verwendet werden sollen:

#!ls ABDA2019/testdaten/cryptominuteresolution

!unzip ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
!mv btcusd.csv ABDA2019/testdaten/cryptominuteresolution/btcusd.csv

Archive:  ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
  inflating: btcusd.csv              


In [0]:
# Das ausführen, wenn eine importierte Datei verwendet werden soll:

#!ls ABDA2019
#!unzip ABDA2019/cryptores.zip    #Dateipfad noch angeben

In [0]:
# Spark einbinden

#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install findspark

#!pip3 install ta-lib --force-reinstall wheel datascience urllib3 kaggle
!pip install ts ts-flint

import findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

#spark = SparkSession.builder.config('spark.driver.extraClassPath', '/path/to/postgresql.jar').getOrCreate()

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Daten auf Interpolation vorbereiten

In [0]:
###############################################################
#
#  Alle csv Dateien einlesen
#
###############################################################

allData = []       
allFilenames = []

directory = "ABDA2019/testdaten/cryptominuteresolution"   #Hier Pfad auf das ändern was man auslesen möchte

import os

for filename in os.listdir(directory):
  if filename.endswith(".csv"):
    print(filename)
    actualFile = directory + "/" 
    actualFile = actualFile + filename
    allData.append(spark.read.csv(actualFile, inferSchema=True, header=True))
    allFilenames.append(filename)

import numpy as np
data = np.asarray(allData)
filenames = np.asarray(allFilenames)

#data[1].show

In [6]:
###############################################################
#
#  Die CSV Dateien für Interpolation aufbereiten
#
###############################################################

modifiedData = []

for i in range (0, len(data)):
  # Erstmal in richtigen Timestamp umwandeln
  tableOne = data[i]
  tableOne.createOrReplaceTempView("viewOne")
  basis_daten = spark.sql("SELECT time, open, close, high, low, volume, CAST(time/1000 AS Timestamp) AS the_date FROM viewOne")
 
  # Timestamp in richtiges Datum umwandeln
  basis_daten.createOrReplaceTempView("oneday")
  dftemp = spark.sql("SELECT time, open, close, high, low, volume, CAST(the_date AS DATE) FROM oneday ORDER BY time")

  #End of the Day bestimmen
  dftemp.createOrReplaceTempView("dates")
  relevantVals = spark.sql("SELECT m1.* FROM dates m1 LEFT JOIN dates m2 ON(m1.the_date = m2.the_date AND m1.time < m2.time) WHERE m2.time IS NULL")
  relevantVals = relevantVals.orderBy('the_date')
  modifiedData.append(relevantVals)

#Daten wieder in einen Array umwandeln
modifiedData = np.asarray(modifiedData)
modifiedData[1].show()

+-------------+--------------------+--------------------+--------------------+--------------------+------------------+----------+
|         time|                open|               close|                high|                 low|            volume|  the_date|
+-------------+--------------------+--------------------+--------------------+--------------------+------------------+----------+
|1558386180000|             1.36E-4|            1.354E-4|             1.36E-4|            1.354E-4|            6100.0|2019-05-20|
|1558468800000|           1.3349E-4|           1.3519E-4|           1.3519E-4|           1.3349E-4|     1832.18285925|2019-05-21|
|1558557600000|           1.4275E-4|           1.4275E-4|           1.4275E-4|           1.4275E-4|1548.9725829000001|2019-05-22|
|1558646820000|           1.4061E-4|           1.4061E-4|           1.4061E-4|           1.4061E-4|             550.9|2019-05-23|
|1558739640000|           1.5249E-4|           1.5249E-4|           1.5249E-4|           1

# Interpolation

In [0]:
# Ausführen, falls es den Interpolationsordner noch nicht gibt

!mkdir ABDA2019/Interpolation

In [0]:
###############################################################################
#
# Methoden für die Interpolation
#
###############################################################################

def getPolVal (xbefore, ybefore, xafter, yafter, x):
  y = ybefore + ((float(x-xbefore)/float(xafter-xbefore)) * (yafter - ybefore))
  return y

from datetime import timedelta, date, datetime

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
      yield start_date + timedelta(n)

In [0]:
###############################################################################
#
# Die Interpolation selbst
#
###############################################################################

for ii in range (0, len(modifiedData)):
  # Tabelle kopieren, weil bei neue Datensätze hinzukommen
  duplicateData = modifiedData[ii]
  #duplicateData.show()

  duplicateDataCollect = duplicateData.collect()
  if (len(duplicateDataCollect)==0): continue

  firstdate = duplicateDataCollect[0].the_date
  lastdate = duplicateDataCollect[-1].the_date

  i = 0
  for single_date in daterange(firstdate, lastdate):
    print(single_date.strftime("%Y-%m-%d"))

    # zweiten Listeneintrag bestimmen, weil sobald ein Tag gegeben ist wird i hochgestzt
    dayTwo = duplicateDataCollect[i]

    # Gibt es diesen Tag nicht beginnt Interpolation
    if(dayTwo.the_date != single_date):   

      # vorheringen Tag bestimmen
      dayOne = duplicateDataCollect[i-1]

      # Zeitstempel bestimmen
      timest = datetime.combine(single_date, datetime.min.time())
      import time 
      timest = time.mktime(timest.timetuple()) * 1000

      # Open Course bestimmen
      from decimal import Decimal
      openCourse = getPolVal(dayOne.time,dayOne.open,dayTwo.time,dayTwo.open,timest)
      #openCourse = '%.3E' % Decimal(openCourse)                                  
      openCourse = format(openCourse, "10.3E")

      # Closing Course bestimmen
      closeCourse = getPolVal(dayOne.time,dayOne.close,dayTwo.time,dayTwo.close,timest) 
      closeCourse = format(closeCourse, "10.3E")

      # High bestimmen
      highCourse = getPolVal(dayOne.time,dayOne.high,dayTwo.time,dayTwo.high,timest) 
      highCourse = format(highCourse, "10.3E")

      # Low bestimmen
      lowCourse = getPolVal(dayOne.time,dayOne.low,dayTwo.time,dayTwo.low,timest) 
      lowCourse = format(lowCourse, "10.3E")

      # volume bestimmen
      volumeCourse = getPolVal(dayOne.time,dayOne.volume,dayTwo.time,dayTwo.volume,timest) 
      volumeCourse = format(volumeCourse, "10.3E")

      # Neue Zeile erstellen und hinzufügen
      newRow = spark.createDataFrame([(timest, openCourse, closeCourse, highCourse, lowCourse, volumeCourse ,single_date)])
      modifiedData[ii] = modifiedData[ii].union(newRow)

    # Es gab den Eintrag, also nächsten Eintag ansehen  
    else:
      i = i+1

  modifiedData[ii] = modifiedData[ii].orderBy('the_date')
  print(filenames[ii])
  print(ii)
  filestring = "ABDA2019/Interpolation/"+filenames[ii] 
  modifiedData[ii].toPandas().to_csv(filestring)       
  
  #modifiedData[ii].coalesce(1).write.option("header", "true").mode("overwrite").csv(filestring)


In [0]:
###############################################################################
#
# Daten zippen, um sie runterladen zu können
#
###############################################################################

!zip -r ABDA2019/Inter_down3.zip ABDA2019/Interpolation

from google.colab import files
files.download("ABDA2019/Inter_down3.zip")